In [36]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pymongo
from pprint import pprint

In [41]:
#urls to scrape
nasamarsurl = "https://mars.nasa.gov/news/"
nasajplurl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
sfurl = "https://space-facts.com/mars/"
usgsurl = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# connect to the web with splinter
driverpath = {'executable_path':'Resources/chromedriver.exe'}

#connect to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#database paths
nasamars_collection = client.mission_to_mars.nasamars
jplimages_collection = client.mission_to_mars.jplimages
spacefacts_collection = client.mission_to_mars.spacefacts
usgsimages_collection = client.mission_to_mars.usgsimages


In [7]:
soup = bs(test, 'html.parser')
content = (soup
           .find('div', class_="col-span-3"))
           #.find_all('div', class_= 'list_text'))
#display(soup)
display(soup)

NameError: name 'test' is not defined

In [6]:
browser = Browser('chrome',**driverpath, headless=False)
browser.visit(nasajplurl)
browser.check("Galleries")
rawdata = browser.html
browser.quit()

ElementDoesNotExist: no elements could be found with name "Galleries"

In [3]:
def pull_data(scrapeurl):

    browser = Browser('chrome',**driverpath, headless=False)
    browser.visit(scrapeurl)
    browser.find_by_name('Galleries')
    rawdata = browser.html
    browser.quit()

    return rawdata

In [4]:
def nasamars_scraper(scrapeurl,collection):
    '''
    
    '''
    #scrape NASA Mars News articles
    def pull_data(scrapeurl):
       
        browser = Browser('chrome',**driverpath, headless=False)
        browser.visit(scrapeurl)
        rawdata = browser.html
        browser.quit()
        
        return rawdata
    
    def clean_data(rawdata,collection):
        # clear database to prevent duplicate entries
        collection.drop()

        #parse the sites html code into a searchable list
        soup = bs(rawdata, 'html.parser')

        # put the html structure for each article into a list
        content = (soup
                    .find('section', class_='grid_gallery module list_view')
                    .find_all('div', class_= 'list_text'))

        for entry in content:
        # iterate through the list and return desired attributes for each article
            try:
                #extract desired attributes
                title = (entry.find('a').text)
                summary = (entry.find('div', class_= 'article_teaser_body').text)
                date = (entry.find('div', class_ = 'list_date').text)
                url = entry.find('a','href')

                #save to mongoDB
                document = {
                    'title': title,
                    'summary': summary,
                    'date': date,
                    'url': url
                }
    
                collection.insert_one(document)

            except Exception as error:
                print(error)
                
    rawdata = pull_data(scrapeurl)
    clean_data(rawdata,collection)

In [5]:
nasamars_scraper(nasamarsurl,nasamars_collection)

In [8]:
def usgs_scraper(scrapeurl,collection):
    '''
    
    '''
    #scrape usgs for links to hi resolution images
    def pull_data(scrapeurl):
       
        browser = Browser('chrome',**driverpath, headless=False)
        browser.visit(scrapeurl)
        rawdata = browser.html
        browser.quit()
        
        return rawdata
    
    def clean_data(rawdata,collection):
        # clear database to prevent duplicate entries
        collection.drop()

        #parse the sites html code into a searchable list
        soup = bs(rawdata, 'html.parser')

        # put the html structure for each article into a list
        content = (soup
                    .find('section', class_='grid_gallery module list_view')
                    .find_all('div', class_= 'list_text'))

        for entry in content:
        # iterate through the list and return desired attributes for each article
            try:
                #extract desired attributes
                title = (entry.find('a').text)
                summary = (entry.find('div', class_= 'article_teaser_body').text)
                date = (entry.find('div', class_ = 'list_date').text)
                url = entry.find('a','href')

                #save to mongoDB
                document = {
                    'title': title,
                    'summary': summary,
                    'date': date,
                    'url': url
                }
                collection.insert_one(document)

            except Exception as error:
                print(error)
                
    rawdata = pull_data(scrapeurl)
    clean_data(rawdata,collection)

In [42]:
browser = Browser('chrome',**driverpath, headless=False)
browser.visit(usgsurl)
rawdata = browser.html

In [65]:
soup = bs(rawdata, 'html.parser')
links = soup.find_all('div', class_='item')
links[0].a['href']
test = []
[test.append(item.a['href']) for item in links]
test[0]

'/search/map/Mars/Viking/cerberus_enhanced'

In [74]:
browser = Browser('chrome',**driverpath, headless=False)
baseurl ="http://astrogeology.usgs.gov"

browser.visit(baseurl + test[0])
testdata = browser.html

In [80]:
soup = bs(testdata, 'html.parser')
highreslink = soup.find('div', class_='downloads').a['href']

highreslink

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

In [34]:
def sf_scraper(scrapeurl,collection):
    """
    
    """
    # retrieve data from source and transform into a dictionary
    def extract_data(scrapeurl):
        # extract datatables from url
        rawdata = pd.read_html(scrapeurl)
        
        #transform first table into a dictionary
        sf_dict = {}
        for row,column in rawdata[0].iterrows():
            sf_dict[column[0]] = column[1]
            
        return sf_dict
    
    # save dictionary to mongoDB
    def load_data(document,collection):    
        collection.insert_one(document)
        
    sf_dict = extract_data(scrapeurl)
    load_data(sf_dict,collection)
   

In [35]:
sf_scraper(sfurl,spacefacts_collection)

In [6]:
rawdata = pd.read_html(sfurl)
marstable = rawdata[0]
marstable

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [29]:
sf_dict = {}
for row,column in rawdata[0].iterrows():
    sf_dict[column[0]] = column[1]


In [30]:
sf_dict

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.38 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-87 to -5 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}